What 's new ?

* notion of teacher
* teacher oriented approach for reducing space and having a more teacher oriented-solution

In [5]:
import numpy as np

In [6]:
timetable_constraints = {
  "nbr_of_days": 5,
  "nbr_of_periods_per_day": 4,
  "classes_data": {
    "groups": ["A", "B", "C"],
    "teachers": [
       {
          "name": "math_teacher",
          "subjects_groups_period": {
            "math": {
              "A": 4,
              "B": 4,
              "C": 4,
            },
            "english": {
              "A": 0,
              "B": 0,
              "C": 0,
            },
            "science": {
              "A": 0,
              "B": 0,
              "C": 0,
            },
          }
        },
        {
          "name": "english_teacher",
          "subjects_groups_period": {
            "math": {
              "A": 0,
              "B": 0,
              "C": 0,
            },
            "english": {
              "A": 4,
              "B": 4,
              "C": 4,
            },
            "science": {
              "A": 0,
              "B": 0,
              "C": 0,
            },
          }
        },
        {
          "name": "science_teacher",
          "subjects_groups_period": {
            "math": {
              "A": 0,
              "B": 0,
              "C": 0,
            },
            "english": {
              "A": 0,
              "B": 0,
              "C": 0,
            },
            "science": {
              "A": 2,
              "B": 2,
              "C": 2,
            },
          }
        },

    ],
    "subjects": ["math", "english", "science"],
  }
}

timetable_constraints["nbr_of_teachers"] = len(timetable_constraints["classes_data"]["teachers"])
timetable_constraints["nbr_of_subjects"] = len(timetable_constraints["classes_data"]["subjects"])

# give each teacher the number of periods he/she can teach
for teacher in timetable_constraints["classes_data"]["teachers"]:
  teacher["nbr_of_periods"] = 0
  subjects_groups_period_array_map = {}
  arr = {}
  for subject in teacher["subjects_groups_period"]:
    for group in teacher["subjects_groups_period"][subject]:
      teacher["nbr_of_periods"] += teacher["subjects_groups_period"][subject][group]
      subjects_groups_period_array_map[f"{group}__{subject}"] = len(arr)
      arr[f"{group}__{subject}"] = teacher["subjects_groups_period"][subject][group]
  teacher["subjects_groups_period_array"] = arr
  teacher["subjects_groups_period_array_map"] = subjects_groups_period_array_map

# slots by teacher
timetable_constraints["slots"] = 0
for teacher in timetable_constraints["classes_data"]["teachers"]:
  timetable_constraints["slots"] += teacher["nbr_of_periods"]

# slots by timetable
timetable_constraints["time_slots"] = timetable_constraints["nbr_of_days"] * timetable_constraints["nbr_of_periods_per_day"]

timetable_constraints["teachers_slots_array"] = []
for teacher in timetable_constraints["classes_data"]["teachers"]:
  timetable_constraints["teachers_slots_array"].append(teacher["nbr_of_periods"])

timetable_constraints

{'nbr_of_days': 5,
 'nbr_of_periods_per_day': 4,
 'classes_data': {'groups': ['A', 'B', 'C'],
  'teachers': [{'name': 'math_teacher',
    'subjects_groups_period': {'math': {'A': 4, 'B': 4, 'C': 4},
     'english': {'A': 0, 'B': 0, 'C': 0},
     'science': {'A': 0, 'B': 0, 'C': 0}},
    'nbr_of_periods': 12,
    'subjects_groups_period_array': {'A__math': 4,
     'B__math': 4,
     'C__math': 4,
     'A__english': 0,
     'B__english': 0,
     'C__english': 0,
     'A__science': 0,
     'B__science': 0,
     'C__science': 0},
    'subjects_groups_period_array_map': {'A__math': 0,
     'B__math': 1,
     'C__math': 2,
     'A__english': 3,
     'B__english': 4,
     'C__english': 5,
     'A__science': 6,
     'B__science': 7,
     'C__science': 8}},
   {'name': 'english_teacher',
    'subjects_groups_period': {'math': {'A': 0, 'B': 0, 'C': 0},
     'english': {'A': 4, 'B': 4, 'C': 4},
     'science': {'A': 0, 'B': 0, 'C': 0}},
    'nbr_of_periods': 12,
    'subjects_groups_period_array'

In [7]:
class Timetable:
  def __init__(self, npArray, constraints):
    self.constraints = constraints
    self.timetable = npArray
    teachers_table = np.array_split(self.timetable, np.cumsum(self.constraints["teachers_slots_array"]))
    teachers_table_bygroup_ = []
    for i, teacher_table in enumerate(teachers_table):
      teachers_table_bygroup_.append(np.array_split(teacher_table, np.cumsum(self.constraints["classes_data"]["teachers"][i]["subjects_groups_period_array"].values())))

    mapped_teachers_table = {} # {"teacher": ["group__subject": [time_slots]]}
    mapped_group_table = {} # {class: {teacher: [time_slots]}}
    for i, teacher in enumerate(self.constraints["classes_data"]["teachers"]):
      mapped_teachers_table[teacher["name"]] = {}
      for subject in teacher["subjects_groups_period"]:
        for group in teacher["subjects_groups_period"][subject]:
          mapped_teachers_table[teacher["name"]][f"{group}__{subject}"] = teachers_table_bygroup_[i][teacher["subjects_groups_period_array_map"][f"{group}__{subject}"]]
          
          if not teacher in mapped_group_table[group] :
            mapped_group_table[teacher["name"]] = {}
          
          mapped_group_table[group][teacher["name"]] = teachers_table_bygroup_[i][teacher["subjects_groups_period_array_map"][f"{group}__{subject}"]]

    self.mapped_teachers_table = mapped_teachers_table
    self.mapped_group_table = mapped_group_table

    self.teachers_table = teachers_table

  def toNpArray(self):
    return self.timetable.flatten()

  def isValid(self):
    pass

  def fitness(self):
    # check if the timetable is valid
    # by checking if teacher does not have a repeated time slot
    score = 0
    max_score = self.constraints["slots"]

    for teacher in self.mapped_teachers_table:
      teacher_slots = []
      for subject in self.mapped_teachers_table[teacher]:
        teacher_slots.append(self.mapped_teachers_table[teacher][subject])
      score += len(np.unique(np.concatenate(teacher_slots)))**2
    
    return Math.sqrt(score) / max_score

  def plot(self):
    pass

  def print(self):
    print("Timetable : ")
    print(self.mapped_group_table)
    print(self.mapped_teachers_table)
    pass

In [8]:
import pygad

In [9]:
def fitness_func(ga_instance, solution, solution_idx):
    timeTable = Timetable(solution, timetable_constraints)
    fitness = timeTable.fitness()
    return fitness

In [11]:
fitness_function = fitness_func

num_generations = 2000
num_parents_mating = 50
sol_per_pop = 200

# gene_space = [ 
#               { 
#                 "low" : 0, 
#                 "high": timetable_constraints["time_slots"],
#                 "step": 1
#               } 
#               for _ in range(timetable_constraints["nbr_of_groups"] * timetable_constraints["nbr_of_days"] * timetable_constraints["nbr_of_periods_per_day"])
#              ]
num_genes = timetable_constraints["slots"]
gene_type = [int for _ in range(num_genes)]

parent_selection_type = "sss"
keep_parents = 1
crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

In [12]:
from tqdm import tqdm
progress_bar = tqdm(total=num_generations, bar_format='{l_bar}{bar:20}{r_bar}{bar:-10b}')

def on_gen(ga_instance):
    progress_bar.update(1)
    progress_bar.set_description_str("Fitness={fitness:.4f}\t".format(fitness=ga_instance.best_solution()[1]))
    # print("Fitness of the best solution :", ga_instance.best_solution()[1])

def on_stop(ga_instance, last_generation_fitness):
    progress_bar.close()
    print("Fitness of the best solution :", ga_instance.best_solution()[1])

ga_instance = pygad.GA(num_generations=num_generations,
                       on_generation=on_gen,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                    #    gene_space=gene_space,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                      #  mutation_probability=[0.5, 0.1],
                       gene_type=gene_type,
                       num_genes=num_genes,
                       on_stop=on_stop
                      )

ga_instance.run()
ga_instance.plot_fitness()

  0%|                    | 0/2000 [00:00<?, ?it/s]slice indices must be integers or None or have an __index__ method
Traceback (most recent call last):
  File "c:\Users\Amine\Desktop\projects\school_timetable_generator\lib\site-packages\pygad\pygad.py", line 1688, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
  File "C:\Users\Amine\AppData\Local\Temp\ipykernel_16896\4077747710.py", line 2, in fitness_func
    timeTable = Timetable(solution, timetable_constraints)
  File "C:\Users\Amine\AppData\Local\Temp\ipykernel_16896\1493193656.py", line 8, in __init__
    teachers_table_bygroup_.append(np.array_split(teacher_table, np.cumsum(self.constraints["classes_data"]["teachers"][i]["subjects_groups_period_array"].values())))
  File "c:\Users\Amine\Desktop\projects\school_timetable_generator\lib\site-packages\numpy\lib\shape_base.py", line 782, in array_split
    sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))
TypeError: slice indices must be integers or None or 

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
progress_bar.close()

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [2 0 1 -1 2 0 2 1 -1 0 1 2 -1 0 1 2 1 1 -1 2 0 -1 0 1 2 2 2 2 -1 0 -1 1 1
 2 1 -1 0 2 1 1 2 -1 1 2 1 0 0 2 1 -1 0 2 1 1 -1 2 0 -1 1 2]
Fitness value of the best solution = 0.9200000000001001


In [ ]:
timetable = Timetable(solution, timetable_constraints)

In [ ]:
table = timetable.timetable
table = [[["-" if period == -1 else timetable_constraints["subject_names"][period] for period in day] for day in group] for group in table]

import pandas as pd
table_pd = pd.DataFrame(table[2])
# add day and period columns
table_pd.insert(0, "Day", ["Day "+str(i) for i in range(1, timetable_constraints["nbr_of_days"]+1)])
#rotate the table
table_pd = table_pd.transpose()
table_pd

,0,1,2,3,4
Day,Day 1,Day 2,Day 3,Day 4,Day 5
0,Anglais,Français,Français,Français,Maths
1,-,Maths,-,Français,-
2,Français,Maths,Maths,-,Français
3,Anglais,Anglais,Anglais,Anglais,Anglais
